In [1]:
import os
import sys

# Get the absolute path to the project's root directory and navigate from there.
# This assumes the notebook is in the 'notebooks' folder.
# os.path.abspath(__file__) works in .py files, but in notebooks we use os.getcwd()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the 'src' directory to Python's path
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# Now, you can import your modules from the 'src' directory
import torch
from utils import calculate_delta_parameters

# --- Example Usage ---
print("Successfully imported 'calculate_delta_parameters' using sys.path!")

# (The rest of the example code remains the same)
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(10, 20)
    def forward(self, x):
        return self.linear1(x)

pretrained_model = SimpleModel()
finetuned_model = SimpleModel()
with torch.no_grad():
    finetuned_model.linear1.weight += 0.1

delta_params = calculate_delta_parameters(pretrained_model, finetuned_model)
print("\nDelta for 'linear1.weight':\n", delta_params['linear1.weight'])

Successfully imported 'calculate_delta_parameters' using sys.path!

Delta for 'linear1.weight':
 tensor([[-0.0478, -0.0470, -0.0722, -0.0315,  0.2104, -0.1691,  0.2968,  0.3209,
          0.1526,  0.0430],
        [ 0.5231, -0.1198,  0.3351,  0.2915,  0.4942,  0.3196, -0.4010,  0.2483,
         -0.1163,  0.1887],
        [ 0.2178,  0.2108,  0.5191,  0.4404, -0.0462, -0.4764,  0.3557,  0.2989,
         -0.1637, -0.2200],
        [ 0.0231, -0.1113,  0.1239, -0.2835, -0.3367, -0.0649,  0.0432, -0.3152,
         -0.0645, -0.4406],
        [ 0.6554, -0.1883,  0.5113,  0.0805,  0.0789,  0.1367, -0.0030,  0.4068,
          0.0918,  0.1403],
        [ 0.1779,  0.7106,  0.1371, -0.5205,  0.1100, -0.0544, -0.4055, -0.0362,
          0.4099, -0.0933],
        [ 0.0841,  0.5952,  0.2625,  0.5957, -0.1909,  0.2337,  0.3199, -0.3877,
         -0.2262, -0.0243],
        [ 0.0921,  0.2066,  0.3811,  0.2915,  0.0121,  0.2504,  0.0858,  0.0658,
          0.5264,  0.0329],
        [ 0.6123,  0.1868,  0.0

In [ ]:
# Import the new function from our compression module
from core.compression import tensor_to_patches

# --- Example Usage for Patchlization ---

# Let's take the delta tensor of one layer
layer_delta = delta_params['linear1.weight']
print(f"Original tensor shape: {layer_delta.shape}")

# Define a patch size
# In the paper, they use sizes like 8, 16, 32, etc.
# For our small example, let's use a smaller size.
p_size = 4 

# Convert the tensor to patches
patches_tensor = tensor_to_patches(layer_delta, patch_size=p_size)

print(f"Patch size: {p_size}x{p_size}")
print(f"Resulting patches tensor shape: {patches_tensor.shape}")
print(f"The first patch:\n {patches_tensor[0]}")

In [ ]:
# Import the new function as well
from core.compression import tensor_to_patches, calculate_importance_scores

# --- Example Usage for Importance Assessment ---

# We use the 'patches_tensor' from the previous step
importance_scores = calculate_importance_scores(patches_tensor)

print(f"\nShape of the importance scores tensor: {importance_scores.shape}")
print(f"Number of scores matches number of patches: {importance_scores.shape[0] == patches_tensor.shape[0]}")
print(f"First 5 importance scores:\n {importance_scores[:5]}")

In [ ]:
# Import the new function
from core.compression import tensor_to_patches, calculate_importance_scores, allocate_bit_widths

# --- Example Usage for Bit-width Allocation ---

# Define the allocation strategy.
# The paper uses a 50% 2-bit and 50% 0-bit (sparsification) setup[cite: 273].
# This is equivalent to a 1-bit compression ratio[cite: 273].
bit_strategy = [(2, 0.5), (0, 0.5)]

# Allocate bit-widths based on the importance scores
allocated_bits = allocate_bit_widths(importance_scores, bit_strategy)

print(f"\nShape of the bit allocation tensor: {allocated_bits.shape}")

# Verification
num_2_bits = torch.sum(allocated_bits == 2).item()
num_0_bits = torch.sum(allocated_bits == 0).item()
total_patches = len(allocated_bits)

print(f"Total patches: {total_patches}")
print(f"Patches assigned 2 bits: {num_2_bits} (~{num_2_bits/total_patches:.0%})")
print(f"Patches assigned 0 bits: {num_0_bits} (~{num_0_bits/total_patches:.0%})")

In [ ]:
# Import the new function
from core.compression import tensor_to_patches, calculate_importance_scores, allocate_bit_widths, dct_and_quantize_patches

# --- Example Usage for DCT and Quantization ---

# We use the 'patches_tensor' and 'allocated_bits' from previous steps.
compressed_data, min_values, max_values = dct_and_quantize_patches(patches_tensor, allocated_bits)

print("\n--- Compression Results ---")
print(f"Number of compressed patches: {len(compressed_data)}")
print(f"Shape of the first compressed patch: {compressed_data[0].shape}")
print(f"Data type of compressed patch: {compressed_data[0].dtype}")

print(f"\nShape of min values tensor: {min_values.shape}")
print(f"Shape of max values tensor: {max_values.shape}")

# Let's inspect a 2-bit patch vs a 0-bit patch
patch_2bit_index = (allocated_bits == 2).nonzero(as_tuple=True)[0][0].item()
patch_0bit_index = (allocated_bits == 0).nonzero(as_tuple=True)[0][0].item()

print(f"\nExample of a 2-bit quantized patch (index {patch_2bit_index}):\n{compressed_data[patch_2bit_index]}")
print(f"Its min/max range: {min_values[patch_2bit_index].item():.4f} / {max_values[patch_2bit_index].item():.4f}")

print(f"\nExample of a 0-bit quantized patch (index {patch_0bit_index}):\n{compressed_data[patch_0bit_index]}")
print(f"Its min/max range (should be the same): {min_values[patch_0bit_index].item():.4f} / {max_values[patch_0bit_index].item():.4f}")

In [ ]:

# Import the new functions
from core.decompression import dequantize_and_idct_patches, patches_to_tensor, final_rescale
import torch.nn.functional as F

# --- Example Usage for Decompression ---

# 1. De-quantize and apply Inverse DCT
reconstructed_patches = dequantize_and_idct_patches(compressed_data, min_values, max_values, allocated_bits)

# 2. Reassemble patches into a single tensor
# We need the original shape of the delta tensor before padding
original_delta_shape = layer_delta.shape
reconstructed_delta = patches_to_tensor(reconstructed_patches, original_delta_shape, p_size)

# 3. Apply the final rescaling step
final_reconstructed_delta = final_rescale(layer_delta, reconstructed_delta)

print("\n--- Decompression Results ---")
print(f"Shape of original delta tensor: {layer_delta.shape}")
print(f"Shape of reconstructed delta tensor: {final_reconstructed_delta.shape}")

# Calculate the Mean Squared Error to measure reconstruction quality
mse = F.mse_loss(layer_delta, final_reconstructed_delta)
print(f"\nMean Squared Error (MSE) between original and reconstructed: {mse.item():.8f}")

In [ ]:
# Import the main pipeline function
# Note: To import from 'pipeline.py', we might need to add an __init__.py file
# in the 'src' directory if it doesn't exist.
from pipeline import compress_model

# --- Example Usage for the Full Compression Pipeline ---

# Define the patch size and bit strategy
p_size = 4
bit_strategy = [(2, 0.5), (0, 0.5)]

# Use the models we created earlier
# In a real scenario, these would be large, loaded models.
compressed_data = compress_model(pretrained_model, finetuned_model, p_size, bit_strategy)

print("\n--- Compressed Data Summary ---")
print(f"Compressed data contains {len(compressed_data)} layers.")
print(f"Keys for the first compressed layer ('linear1.weight'): {compressed_data['linear1.weight'].keys()}")

In [ ]:
# ... (previous code) ...
import copy
from pipeline import compress_model, decompress_model

# --- Full Pipeline Example ---

# 1. Compress the model (using the updated function)
p_size = 4
bit_strategy = [(2, 0.5), (0, 0.5)]
compressed_data = compress_model(pretrained_model, finetuned_model, p_size, bit_strategy)

# 2. Decompress the model
reconstructed_finetuned_model = decompress_model(pretrained_model, compressed_data)

# --- Final Verification ---
# Let's compare the parameters of the original fine-tuned model and our reconstructed one.
original_params = finetuned_model.state_dict()['linear1.weight']
reconstructed_params = reconstructed_finetuned_model.state_dict()['linear1.weight']

mse = F.mse_loss(original_params, reconstructed_params)
print(f"\n--- Final Verification ---")
print(f"MSE between original fine-tuned model and reconstructed model: {mse.item():.8f}")